In [1]:
import os
import math
import wandb
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
import pandas as pd
from collections import OrderedDict
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def gen_ue_cluster(UE_sort):# UE를 유클리드 distance 기준으로 클러스터링한 리스트를 반환하는 함수
    UE_cluster = []
    UE_group = []
    UE_distance = [] # 2 UE 간의 거리 리스트
    UE_len = len(UE_sort)
    avg_distance = 0
    UE_sort_keys = list(UE_sort.keys())
    for i in range(0, UE_len - 1):
        x1, y1 = i + 1, UE_sort[UE_sort_keys[i]]
        x2, y2 = i+2, UE_sort[UE_sort_keys[i+1]]
        distance = (((x2 - x1) ** 2) + ((y2 - y1) ** 2)) ** 0.5
        avg_distance += distance
        UE_distance.append((UE_sort_keys[i], UE_sort_keys[i+1],distance))
    avg_distance = avg_distance/len(UE_sort)
    
    for i, x in enumerate(UE_distance):
        if UE_distance[-1][0] == x[0]:
            UE_group.append(x[0])
            if x[2] > avg_distance:
                UE_cluster.append(UE_group.copy())
                UE_group.clear()
                UE_group.append(x[1])
                UE_cluster.append(UE_group.copy())
            else:
                UE_group.append(x[1])
                UE_cluster.append(UE_group.copy())
        else:
            if x[2] > avg_distance:
                UE_group.append(x[0])
                UE_cluster.append(UE_group.copy())
                UE_group.clear()
            else:
                UE_group.append(x[0])
            
    
    return UE_cluster
    
def gap(weights): # Global average pooling
    if len(weights.shape) == 4:
        row = np.mean(weights, axis=1)
        result = np.mean(row, axis=0)
    elif len(weights.shape) == 2:
        result = np.mean(weights, axis=0)
    else:
        raise print('가중치 잘못됨')

    return result


def split_ue_group(UE_weights, UE_NUM): # 각 레이어별 가중치 평균보다 높은지 낮은지 나누는 함수
    layers = UE_weights[0].keys()
    concat_weight = {}
    concat_mean_weight = {}

    for layer in layers:
        total = np.zeros(
            (UE_NUM, UE_weights[0][layer][0].shape[-2:][0], UE_weights[0][layer][0].shape[-2:][1]))
        for i, UE in enumerate(UE_weights):
            total[i] = gap(UE[layer][0])
        concat_weight[layer] = total

    for layer in concat_weight.keys():
        concat_mean_weight[layer] = np.mean(concat_weight[layer], axis=0)

    UE_high_low = {}
    for layer in concat_weight.keys():
        true_cnt_list = []
        for x in range(UE_NUM):
            high_low = concat_weight[layer][x] > concat_mean_weight[layer]
            if len(high_low[high_low == True]) >= len(high_low[high_low == False]):
                true_cnt_list.append(True)
            else:
                true_cnt_list.append(False)
        UE_high_low[layer] = true_cnt_list
        
    result = [0 for _ in range(UE_NUM)]
    for layer in UE_high_low.keys():
        for i, x in enumerate(UE_high_low[layer]):
            if x == True:
                result[i] += 1

    d = dict()
    
    for i, x in enumerate(result):
        d[i] = x

    UE_sort = OrderedDict(sorted(d.items(), key=lambda t:t[1]))
    
    return gen_ue_cluster(UE_sort)


def gen_UE_model():
    model = models.Sequential()
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5),
                            strides=1, activation='tanh', input_shape=(32, 32, 1)))
    model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    model.add(layers.Conv2D(filters=16, kernel_size=(
        5, 5), strides=1, activation='tanh'))
    model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='tanh'))
    model.add(layers.Dense(84, activation='tanh'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='SGD',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # lr = 0.01

    return model


def gen_server_model():
    server_model = models.Sequential()
    server_model.add(layers.Conv2D(filters=6, kernel_size=(
        5, 5), strides=1, activation='tanh', input_shape=(32, 32, 1)))
    server_model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    server_model.add(layers.Conv2D(
        filters=16, kernel_size=(5, 5), strides=1, activation='tanh'))
    server_model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    server_model.add(layers.Flatten())
    server_model.add(layers.Dense(120, activation='tanh'))
    server_model.add(layers.Dense(84, activation='tanh'))
    server_model.add(layers.Dense(10, activation='softmax'))

    return server_model

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_test = x_test.reshape(10000, 28, 28, 1) / 255
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
x_test = np.pad(x_test, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
idx = np.argsort(y_test)
x_train_sorted = x_test[idx]
y_train_sorted = y_test[idx]

UE_NUM = 20

UE = []
for _ in range(UE_NUM):
    UE.append({"x_train": [], "y_train": []})

random.seed(45)
seed = random.randint(1,1000000)
print(seed)
print(f'seed : 235511')
tf.random.set_seed(235511) # 100 좋은거, 12: 5 클러스터, 403092, 235511, 130670 : 6 클러스터, 192150 : 20

total = 0

x_eval_dataset = x_test.copy()
y_eval_dataset = y_test.copy()

data_info = []

for i in range(UE_NUM):
    random_start = random.randint(1,50000)
    #     data_num = random.randint(5000, 30000)
    data_num = random.randint(10000, 60000-random_start)
    print(random_start, random_start+data_num)
    data_info.append([random_start, random_start+data_num, random_start+data_num-random_start])
    UE[i]['x_train'] = x_train[random_start:random_start+data_num]
    UE[i]['y_train'] = y_train[random_start:random_start+data_num]

x_train, x_test, y_train, y_test = [], [], [], []
for i in range(UE_NUM):
    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(
        UE[i]['x_train'], UE[i]['y_train'], test_size=0.2, random_state=45)
    x_train.append(x_train_temp)
    x_test.append(x_test_temp)
    y_train.append(y_train_temp)
    y_test.append(y_test_temp)

285083
seed : 235511
27374 53365
16887 32253
19870 40969
1416 16156
31737 42098
7527 36129
7715 38099
48725 58798
40956 52047
8849 36647
48298 58613
13829 51903
4913 55081
26748 41427
45905 56373
2109 23395
21016 39269
23367 58765
47527 58837
6145 43106


In [3]:
for i, x in enumerate(UE):
    print(f"Client {i+1}'s data amount : {len(x['x_train'])}")

Client 1's data amount : 25991
Client 2's data amount : 15366
Client 3's data amount : 21099
Client 4's data amount : 14740
Client 5's data amount : 10361
Client 6's data amount : 28602
Client 7's data amount : 30384
Client 8's data amount : 10073
Client 9's data amount : 11091
Client 10's data amount : 27798
Client 11's data amount : 10315
Client 12's data amount : 38074
Client 13's data amount : 50168
Client 14's data amount : 14679
Client 15's data amount : 10468
Client 16's data amount : 21286
Client 17's data amount : 18253
Client 18's data amount : 35398
Client 19's data amount : 11310
Client 20's data amount : 36961


In [4]:
data_info

[[27374, 53365, 25991],
 [16887, 32253, 15366],
 [19870, 40969, 21099],
 [1416, 16156, 14740],
 [31737, 42098, 10361],
 [7527, 36129, 28602],
 [7715, 38099, 30384],
 [48725, 58798, 10073],
 [40956, 52047, 11091],
 [8849, 36647, 27798],
 [48298, 58613, 10315],
 [13829, 51903, 38074],
 [4913, 55081, 50168],
 [26748, 41427, 14679],
 [45905, 56373, 10468],
 [2109, 23395, 21286],
 [21016, 39269, 18253],
 [23367, 58765, 35398],
 [47527, 58837, 11310],
 [6145, 43106, 36961]]

In [4]:
simulation_num = 1
for num in range(simulation_num):
    wandb.init(project='Federated Learning (KCI)',
               name=f'0930 Cluster Simulation 2', entity='yhkim')
    
    wandb.log({'data info' : data_info}, commit=False)
    save_path = f'0930_simulation_result/2'
    os.mkdir(save_path)
    
    cluster_result = {}

    learning_result_list = []
    for i in range(UE_NUM):
        model = gen_UE_model()

        learning_result_list.append(model.fit(
            x_train[i], y_train[i], batch_size=100, epochs=1, validation_data=(x_test[i], y_test[i])))
        tf.keras.backend.clear_session()

    UE_weights = []

    for model in learning_result_list:
        layer_weights = {}
        for x in model.model.layers:
            if len(x.get_weights()) > 0:
                layer_weights[x.name] = x.get_weights()
        UE_weights.append(layer_weights)

    server_model = gen_server_model()  # FL 서버 모델 생성

    sum_weights = {}

    for i in range(len(list(UE_weights[0].keys()))):
        weight_shape = [0]
        bias_shape = [0]
        for dim in UE_weights[0][list(UE_weights[0].keys())[i]][0].shape:
            weight_shape.append(dim)
        for dim in UE_weights[0][list(UE_weights[0].keys())[i]][1].shape:
            bias_shape.append(dim)
        sum_weights.update({list(UE_weights[0].keys())[i]: {
            'weight': np.empty(weight_shape), 'bias': np.empty(bias_shape)}})

        for UE in UE_weights:
            sum_weights[list(UE.keys())[i]]['weight'] = np.append(
                sum_weights[list(UE.keys())[i]]['weight'], [UE[list(UE.keys())[i]][0]], axis=0)
            sum_weights[list(UE.keys())[i]]['bias'] = np.append(
                sum_weights[list(UE.keys())[i]]['bias'], [UE[list(UE.keys())[i]][1]], axis=0)

    for layer in sum_weights.keys():
        for model_layer in server_model.layers:
            if layer == model_layer.name:
                model_layer.set_weights([np.mean(sum_weights[layer]['weight'], axis=0), np.mean(
                    sum_weights[layer]['bias'], axis=0)])

    server_model.compile(
        optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    result = server_model.evaluate(
        x=x_eval_dataset, y=y_eval_dataset, batch_size=128)
    tf.keras.backend.clear_session()
    print('Initial Round -- test loss, test acc:', result)
    split_result = split_ue_group(UE_weights, UE_NUM)
    print(split_result)

    server_model.save(f'{save_path}/fl_model_gap')

    for round in range(100):  # Communication Round, Global epoch
        learning_result_list = []
        
        for cluster_num, cluster in enumerate(split_result):
            cluster_learning_result_list = []
            cluster_list = []
            for i in cluster:
                cluster_list.append(i)
                
                i = int(i)
                if round == 0:
                    model = tf.keras.models.load_model(
                        f'{save_path}/fl_model_gap')
                    model.compile(optimizer='SGD',
                                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                    cluster_learning_result_list.append(model.fit(
                        x_train[i], y_train[i], batch_size=100, epochs=1, validation_data=(x_test[i], y_test[i])))
                    tf.keras.backend.clear_session()
                else:
                    model = tf.keras.models.load_model(
                        f'{save_path}/cluster{cluster_num}')
                    model.compile(optimizer='SGD',
                                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                    cluster_learning_result_list.append(model.fit(
                        x_train[i], y_train[i], batch_size=100, epochs=1, validation_data=(x_test[i], y_test[i])))
                    tf.keras.backend.clear_session()

            wandb.log({f'cluster{cluster_num}':cluster_list}, commit=False)
            
            UE_weights = []

            for model in cluster_learning_result_list:
                layer_weights = {}
                for x in model.model.layers:
                    if len(x.get_weights()) > 0:
                        layer_weights[x.name] = x.get_weights()
                UE_weights.append(layer_weights)

            server_model = gen_server_model()
            tf.keras.backend.clear_session()

            sum_weights = {}

            for i in range(len(list(UE_weights[0].keys()))):
                weight_shape = [0]
                bias_shape = [0]
                for dim in UE_weights[0][list(UE_weights[0].keys())[i]][0].shape:
                    weight_shape.append(dim)
                for dim in UE_weights[0][list(UE_weights[0].keys())[i]][1].shape:
                    bias_shape.append(dim)
                sum_weights.update({list(UE_weights[0].keys())[i]: {
                    'weight': np.empty(weight_shape), 'bias': np.empty(bias_shape)}})

                for UE in UE_weights:
                    sum_weights[list(UE.keys())[i]]['weight'] = np.append(
                        sum_weights[list(UE.keys())[i]]['weight'], [UE[list(UE.keys())[i]][0]], axis=0)
                    sum_weights[list(UE.keys())[i]]['bias'] = np.append(
                        sum_weights[list(UE.keys())[i]]['bias'], [UE[list(UE.keys())[i]][1]], axis=0)

            for layer in sum_weights.keys():
                for model_layer in server_model.layers:
                    if layer == model_layer.name:
                        model_layer.set_weights([np.mean(sum_weights[layer]['weight'], axis=0), np.mean(
                            sum_weights[layer]['bias'], axis=0)])

            server_model.compile(
                optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            result = server_model.evaluate(
                x=x_eval_dataset, y=y_eval_dataset, batch_size=128)
            print(f'Round {round} -- test loss, test acc: {result}')

            server_model.save(f'{save_path}/cluster{cluster_num}')
            cluster_result[f'cluster{cluster_num}_acc'] = result[1]
            cluster_result[f'cluster{cluster_num}_loss'] = result[0]
            
            wandb.log({f'cluster{cluster_num}_acc':result[1], f'cluster{cluster_num}_loss':result[0]}, commit=False)
        
        wandb.log({'global epoch':round+1})
    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yhkim (use `wandb login --relogin` to force relogin)
C:\Users\user\anaconda3\envs\tensor\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


79/79 [==============================] - 0s 1ms/step - loss: 2.3023 - accuracy: 0.1135
Initial Round -- test loss, test acc: [2.3022775650024414, 0.11349999904632568]
[[18], [8, 12], [1, 5, 6, 10, 13, 16], [0, 3, 7, 9, 14, 15, 19], [4, 11], [2, 17]]
79/79 [==============================] - 0s 1ms/step - loss: 2.3021 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.3020856380462646, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3017 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.301724433898926, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3019 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.301887273788452, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3018 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.301841974258423, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3018 - accuracy: 0.1135
Round 0 -- test loss, 

79/79 [==============================] - 0s 1ms/step - loss: 2.3015 - accuracy: 0.1135
Round 1 -- test loss, test acc: [2.301530599594116, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3015 - accuracy: 0.1135
Round 1 -- test loss, test acc: [2.3015124797821045, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 1 -- test loss, test acc: [2.301400661468506, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3018 - accuracy: 0.1135
Round 2 -- test loss, test acc: [2.3018040657043457, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 2 -- test loss, test acc: [2.301140069961548, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 2 -- test loss, test acc: [2.30139422416687, 0.11349999904632568]
79/79 [==============================] 

79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 3 -- test loss, test acc: [2.301116943359375, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 3 -- test loss, test acc: [2.3010001182556152, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3016 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.3016154766082764, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3008 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.3008384704589844, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.3011133670806885, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.3010058403015137, 0.11349999904632568]
79/79 [============================

79/79 [==============================] - 0s 1ms/step - loss: 2.3008 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.300783157348633, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3006 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.3006482124328613, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3006 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.300579071044922, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3004 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.300408124923706, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 9 -- test loss, test acc: [2.3013556003570557, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3001 - accuracy: 0.1135
Round 9 -- test loss, test acc: [2.300100564956665, 0.11349999904632568]
79/79 [==============================]

...

79/79 [==============================] - 0s 1ms/step - loss: 0.1006 - accuracy: 0.9717
Round 90 -- test loss, test acc: [0.10056270658969879, 0.9717000126838684]
79/79 [==============================] - 0s 1ms/step - loss: 0.0873 - accuracy: 0.9740
Round 90 -- test loss, test acc: [0.08732061088085175, 0.9739999771118164]
79/79 [==============================] - 0s 1ms/step - loss: 0.3517 - accuracy: 0.8972
Round 91 -- test loss, test acc: [0.35171130299568176, 0.8971999883651733]
79/79 [==============================] - 0s 1ms/step - loss: 0.0758 - accuracy: 0.9775
Round 91 -- test loss, test acc: [0.07583582401275635, 0.9775000214576721]
79/79 [==============================] - 0s 1ms/step - loss: 0.1331 - accuracy: 0.9622
Round 91 -- test loss, test acc: [0.13312163949012756, 0.9621999859809875]
79/79 [==============================] - 0s 1ms/step - loss: 0.1218 - accuracy: 0.9671
Round 91 -- test loss, test acc: [0.12176433205604553, 0.9671000242233276]
79/79 [=====================

79/79 [==============================] - 0s 1ms/step - loss: 0.1248 - accuracy: 0.9649
Round 95 -- test loss, test acc: [0.12478999048471451, 0.964900016784668]
79/79 [==============================] - 0s 1ms/step - loss: 0.1146 - accuracy: 0.9685
Round 95 -- test loss, test acc: [0.11458422988653183, 0.968500018119812]
79/79 [==============================] - 0s 1ms/step - loss: 0.0943 - accuracy: 0.9735
Round 95 -- test loss, test acc: [0.09425818175077438, 0.9735000133514404]
79/79 [==============================] - 0s 1ms/step - loss: 0.0826 - accuracy: 0.9754
Round 95 -- test loss, test acc: [0.08255204558372498, 0.9753999710083008]
79/79 [==============================] - 0s 1ms/step - loss: 0.3239 - accuracy: 0.9058
Round 96 -- test loss, test acc: [0.3239179253578186, 0.9057999849319458]
79/79 [==============================] - 0s 1ms/step - loss: 0.0719 - accuracy: 0.9781
Round 96 -- test loss, test acc: [0.07185909897089005, 0.9781000018119812]
79/79 [========================

79/79 [==============================] - 0s 1ms/step - loss: 0.1113 - accuracy: 0.9688
Round 97 -- test loss, test acc: [0.11134210228919983, 0.9688000082969666]
79/79 [==============================] - 0s 1ms/step - loss: 0.0920 - accuracy: 0.9734
Round 97 -- test loss, test acc: [0.09200376272201538, 0.9733999967575073]
79/79 [==============================] - 0s 1ms/step - loss: 0.0808 - accuracy: 0.9758
Round 97 -- test loss, test acc: [0.08084145933389664, 0.9757999777793884]
79/79 [==============================] - 0s 1ms/step - loss: 0.3140 - accuracy: 0.9082
Round 98 -- test loss, test acc: [0.314020037651062, 0.9082000255584717]
79/79 [==============================] - 0s 1ms/step - loss: 0.0704 - accuracy: 0.9785
Round 98 -- test loss, test acc: [0.07042492926120758, 0.9785000085830688]
79/79 [==============================] - 0s 1ms/step - loss: 0.1193 - accuracy: 0.9661
Round 98 -- test loss, test acc: [0.11929097771644592, 0.9660999774932861]
79/79 [=======================

cluster0_acc,0.9096
cluster0_loss,0.30929
cluster1_acc,0.9787
cluster1_loss,0.06974
cluster2_acc,0.9673
cluster2_loss,0.11758
cluster3_acc,0.9698
cluster3_loss,0.1083
cluster4_acc,0.9741
cluster4_loss,0.08988
cluster5_acc,0.9764


cluster0_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▄▄▅▆▇▇▇▇▇██████████
cluster0_loss,█████████████████████▇▆▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
cluster1_acc,▁▁▁▁▁▁▁▂▅▆▇▇▇▇▇█████████████████████████
cluster1_loss,████████▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster2_acc,▁▁▁▁▁▁▁▁▁▁▁▁▃▅▆▆▇▇▇▇▇▇██████████████████
cluster2_loss,█████████████▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster3_acc,▁▁▁▁▁▁▁▁▁▁▁▃▅▆▆▇▇▇▇▇▇███████████████████
cluster3_loss,████████████▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster4_acc,▁▁▁▁▁▁▁▁▁▁▄▅▆▇▇▇▇▇██████████████████████
cluster4_loss,██████████▇▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster5_acc,▁▁▁▁▁▁▁▁▃▄▆▇▇▇▇▇████████████████████████


In [10]:
cluster = [[18], [8, 12], [1, 5, 10, 14, 16], [0, 3, 6, 7, 9, 15, 19], [4, 11, 13], [2, 17]]

In [7]:
for x in data_info:
    print(x)

[27374, 53365, 25991]
[16887, 32253, 15366]
[19870, 40969, 21099]
[1416, 16156, 14740]
[31737, 42098, 10361]
[7527, 36129, 28602]
[7715, 38099, 30384]
[48725, 58798, 10073]
[40956, 52047, 11091]
[8849, 36647, 27798]
[48298, 58613, 10315]
[13829, 51903, 38074]
[4913, 55081, 50168]
[26748, 41427, 14679]
[45905, 56373, 10468]
[2109, 23395, 21286]
[21016, 39269, 18253]
[23367, 58765, 35398]
[47527, 58837, 11310]
[6145, 43106, 36961]


In [19]:
cluster_data_amout_list = list()
for c in cluster:
    data_sum = 0
    print(c)
    for x in c:
        print(data_info[x][2])
        data_sum += data_info[x][2]
    cluster_data_amout_list.append(data_sum)

[18]
11310
[8, 12]
11091
50168
[1, 5, 10, 14, 16]
15366
28602
10315
10468
18253
[0, 3, 6, 7, 9, 15, 19]
25991
14740
30384
10073
27798
21286
36961
[4, 11, 13]
10361
38074
14679
[2, 17]
21099
35398


In [18]:
cluster_data_amout_list

[11310, 61259, 83004, 167233, 63114, 56497]

In [5]:
save_path = f'0930_simulation_result/2'
model = tf.keras.models.load_model(
                        f'{save_path}/cluster1')

In [8]:
model.predict(x_eval_dataset)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-8-8c8e666dc5f0>:1) ]] [Op:__inference_predict_function_1092]

Function call stack:
predict_function
